## Pré-processamento é provavelmente a parte mais importante de ciência dos dados

Ter dados representativos sem atributos faltantes é provavelmente o pote de ouro em ciência dos dados. É muito incomum que os dados do mundo real não apresentem anomalias seja da própria natureza ou sejam anomalias introduzidas no processo de medição e registro da observação (amostra).

Esse notebook é voltado para como tratar dados mais complexos e transformar todas as informações em números que façam sentido para que o modelo seja capaz de traçar a relação entre atributos e classes. A seguir é oferecida uma pequena parcela de um conjunto de dados da empresa Porto Seguro, no qual uma competição foi aberta e os competidores foram desafiados a criar um modelo para prever se uma apólice teria um sinistro registrado ou não, indicando o uso do serviço.

Algumas características sobre o nome das features:
- O nome dos atributos indica o grupo ao qual pertence (ind, reg, car);
- Os prefixos bin e cat indicam atributos binários e categóricos, respectivamente;
- Atributos sem os prefixos citados podem ser ordinais ou contínuos;
- Atributos com -1 indicam dado faltante (missing); e
- A coluna 'target' indica se houve sinistro para apólice ou não.

In [ ]:
import pandas as pd

df = pd.read_csv('porto.csv')
df.describe()

Um cuidado especial que precisa ser tomado ao analisar um conjunto de dados é observar a distribuição dos atributos, por exemplo, pelo comando *describe()*. **Os valores estão em uma mesma escala? Será necessário normalizá-los?** Essas são perguntas quem você deverá se fazer ao trabalhar com os dados.

---

Em sequência, é importante verificar se existem registros duplicados. Registros duplicados em uma mesma classe não são relevantes para a maioria dos métodos, e podem economizar processamento quando removidos. **Já registros duplicados com classes diferentes podem confundir praticamente todos os métodos, é extremamente importante removê-los.**

In [ ]:
print('Antes:', df.shape)
df.drop_duplicates()
print('Depois:', df.shape)

In [ ]:
train = df[:18000]
test  = df[18000:]

display(train.describe())
display(test.describe())

Ao trabalhar com as colunas (atributos), é interessante ter uma organização de que tipo de dado determinado atributo é, e para quais propósitos determinado atributo pode ser usado. Nesse sentido, seguindo o trabalho de https://www.kaggle.com/bertcarremans/data-preparation-exploration, vamos criar metadados para esse conjunto.

In [ ]:
data = []
for f in train.columns:
    # definindo o uso (entre rótulo, id e atributos)
    if f == 'target':
        role = 'target' # rótulo
    elif f == 'id':
        role = 'id'
    else:
        role = 'input' # atributos
         
    # definindo o tipo do dado
    if 'bin' in f or f == 'target':
        level = 'binary'
    elif 'cat' in f or f == 'id':
        level = 'nominal'
    elif train[f].dtype == float:
        level = 'interval'
    elif train[f].dtype == int:
        level = 'ordinal'
        
    # mantem keep como verdadeiro pra tudo, exceto id
    keep = True
    if f == 'id':
        keep = False
    
    # cria o tipo de dado
    dtype = train[f].dtype
    
    # cria dicionário de metadados
    f_dict = {
        'varname': f,
        'role': role,
        'level': level,
        'keep': keep,
        'dtype': dtype
    }
    data.append(f_dict)
    
meta = pd.DataFrame(data, columns=['varname', 'role', 'level', 'keep', 'dtype'])
meta.set_index('varname', inplace=True)

Para visualizar o atributo e todos seus metadados, basta mostrar a variável meta:

In [ ]:
meta

Com essa estrutura de metadados, fica fácil consultar quais colunas quer se manter e que são nominais, por exemplo:

In [ ]:
meta[(meta.level == 'nominal') & (meta.keep)].index

Da mesma forma, seria possível contar os atributos por tipo de uso e dado:

In [ ]:
pd.DataFrame({'count' : meta.groupby(['role', 'level'])['role'].size()}).reset_index()

## Valores faltantes

Conforme já mencionado, os valores faltantes são indicados por -1, então é importante saber quais colunas têm valores faltantes e em qual proporção.

In [ ]:
atributos_missing = []

for f in train.columns:
    missings = train[train[f] == -1][f].count()
    if missings > 0:
        atributos_missing.append(f)
        missings_perc = missings/df.shape[0]
        
        print('Atributo {} tem {} amostras ({:.2%}) com valores faltantes'.format(f, missings, missings_perc))
        
print('No total, há {} atributos com valores faltantes'.format(len(atributos_missing)))

Duas estratégias podem ser optadas aqui: simplesmente remover o atributo ou tentar preenchê-lo de forma sintética. Preencher de forma sintética pode gerar uma falsa distribuição quando o número de atributos faltantes é muito alto. Quando este for o caso, é sempre seguro optar por remover o atributo inteiro. Também é importante lembrar que a estratégia de preenchimento deve ser coerente com o tipo de dado, por exemplo: **dados ordinais não devem ser preenchidos com média, nem dados contínuos com moda.**

In [ ]:
# removendo ps_car_03_cat e ps_car_05_cat que tem muitos valores faltantes
vars_to_drop = ['ps_car_03_cat', 'ps_car_05_cat']
train = train.drop(vars_to_drop, axis=1)
test = test.drop(vars_to_drop, axis=1)
meta.loc[(vars_to_drop),'keep'] = False  # atualiza os metadados para ter como referência (processar o test depois)

In [ ]:
from sklearn.preprocessing import Imputer

media_imp = Imputer(missing_values=-1, strategy='mean', axis=0)
moda_imp = Imputer(missing_values=-1, strategy='most_frequent', axis=0)

train['ps_reg_03'] = media_imp.fit_transform(train[['ps_reg_03']]).ravel()
train['ps_car_12'] = media_imp.fit_transform(train[['ps_car_12']]).ravel()
train['ps_car_14'] = media_imp.fit_transform(train[['ps_car_14']]).ravel()
train['ps_car_11'] = moda_imp.fit_transform(train[['ps_car_11']]).ravel()

test['ps_reg_03'] = media_imp.fit_transform(test[['ps_reg_03']]).ravel()
test['ps_car_12'] = media_imp.fit_transform(test[['ps_car_12']]).ravel()
test['ps_car_14'] = media_imp.fit_transform(test[['ps_car_14']]).ravel()
test['ps_car_11'] = moda_imp.fit_transform(test[['ps_car_11']]).ravel()

Os atributos categóricos podem ser mantidos porque o número de valores faltantes não é expressivo. Inclusive, a estratégia de preenchimento dos **atributos categóricos** é sempre mais complexa. Esses atributos **não se beneficiam de medidas estatísticas** como moda e média, portanto essas medidas não servem para preenchê-los de forma sintética.

---

## One-hot encoding (ou dummy variables)

Depois de ter tratado os dados faltantes, é importante que os dados ordinais tenham representação apropriada para o problema tratado. Se o dado não tem distância ou rankamento entre eles, cada valor de um atributo deve ser representado por um conjunto de atributos de mesma distância. *(Verificar slides desse encontro para que isso fique mais claro)*

Os dados que precisam ser separados em mais dimensões já foram identificados como nominais no pré-processamento. É importante verificar se esses dados têm grande variedade de valores ou não, e aplicar essa separação apenas se for viável. Por exemplo, se um determinado atributo tem 300 valores, isso geraria 300 colunas novas. Isso só se justificaria se fosse uma base realmente grande e se houvesse uma correlação muito alta entre essa variedade de valores e a classe.

In [ ]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index

for f in v:
    dist_values = train[f].value_counts().shape[0]
    print('Atributo {} tem {} valores distintos'.format(f, dist_values))

Vamos optar por manter todos atributos e, portanto, gerar o conjunto de atributos que os mantêm à mesma distância:

In [ ]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index
print('Antes do one-hot encoding tinha-se {} atributos'.format(train.shape[1]))
train = pd.get_dummies(train, columns=v, drop_first=True)
print('Depois do one-hot encoding tem-se {} atributos'.format(train.shape[1]))

test = pd.get_dummies(test, columns=v, drop_first=True)
missing_cols = set( train.columns ) - set( test.columns )
for c in missing_cols:
    test[c] = 0
    
train, test = train.align(test, axis=1)

## Depois de todo pré-processamento...

É hora de verificar se tanto treino como teste têm o mesmo tamanho/formato, e aplicar um modelo de classificação já que esse é um problema desse tipo. Vale lembrar que o tamanho do treino e teste pode variar quando você estiver participando de outras competições ou explorando outros conjuntos de dados.

Isso porque na maioria das competições não se tem o *target* do test. Estima-se uma resposta e submete ao Kaggle, por exemplo, para que ele verifique qual foi o resultado final. Então esse tamanho pode variar em 1 entre treino e teste. No nosso caso, como todos os dados vêm de uma mesma fonte para experimentos, é esperado que tenham a mesma quantidade de atributos ou colunas.

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
X_train = train.drop(['id', 'target'], axis=1)
y_train = train['target']

X_test  = test.drop(['id', 'target'], axis=1)
y_test  = test['target']

from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

model.fit(X_train, y_train)
model.score(X_test, y_test)